In [ ]:
import os, subprocess, time, glob
output_path = 'AI_PICS'
username='a'
password='a'
NGROK ='' 

In [ ]:
Save_In_Google_Drive = 'Models Only'
Everything = 1
Models_only = 2
Nothing = 3
if Save_In_Google_Drive == 'Models Only':
  gMode = Models_only
elif Save_In_Google_Drive == 'Everything':
  gMode = Everything
elif Save_In_Google_Drive == 'Nothing':
  gMode = Nothing
else:
  raise ValueError("Save_In_Google_Drive value invalid.")

In [ ]:
Clear_Log = True
v1_5_inpainting_model = True
v2_depth_model = False
Model_from_URL = ''
Extensions_from_URL = ''

In [ ]:
if gMode != Nothing:
  # connect to google drive
  from google.colab import drive
  drive.mount('/content/drive') 
  output_path = '/content/drive/MyDrive/' + output_path
  if gMode == Everything:
    # install A1111 in AI_PIC
    root = output_path 
  elif gMode == Models_only:
    # install A1111 in Colab
    root = '/content/' 
    !mkdir -p {output_path}/outputs
    !mkdir -p {output_path}/models
    !mkdir -p {output_path}/ESRGAN
    !mkdir -p {output_path}/hypernetworks
  else:
    raise ValueError("Unexpected gMode: %s"%gMode)
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path
  root = '/content/' 

Mounted at /content/drive


In [ ]:
def clear():
    from IPython.display import clear_output; return clear_output()

def fetch_bytes(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        from urllib.request import urlopen 
        return urlopen(url_or_path) 
    return open(url_or_path, 'r')

def packages():
    import sys, subprocess
    return [r.decode().split('==')[0] for r in subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).split()]

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}


In [ ]:
def download_models():
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  print('⏳ Downloading models ...')
  if v1_5_inpainting_model:  
    downloadModel('https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt')
  if v2_depth_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/512-depth-ema.yaml
  if Model_from_URL:
      for m in Model_from_URL.split(','):
        %cd {root}/stable-diffusion-webui/models/Stable-diffusion
        downloadModel(m)
  # download VAEs
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-ema-560000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt

In [ ]:
def installxformers():
  #!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  #%pip install --no-deps -q https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl
  %pip install -q xformers

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print('Already python 3.10. Skip install.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

In [ ]:
def initSaveGoogleDriveModelOnly():
  # Use config file in google drive
  if not os.path.exists(output_path + '/ui-config.json'):
    print("Create new ui-config.json file.")    
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/ui-config.json -O {output_path + '/ui-config.json'}
  if not os.path.exists(output_path + '/config.json'):
    print("Create new config.json file.")    
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/config.json -O {output_path + '/config.json'}

  #!rm /content/stable-diffusion-webui/ui-config.json
  !ln -s {output_path + '/ui-config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path + '/config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path}/outputs
  
  # embeddings folder on Google Drive
  !mkdir -p {output_path}/embeddings  
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # link all models in the models folder
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  models_in_google_drive = glob.glob(output_path + '/models/*')
  print('Models in Google Drive: %s'%models_in_google_drive)
  for f in models_in_google_drive:
    !ln -s {f}

  # link all upscalers in the model folder
  !mkdir -p {root}/stable-diffusion-webui/models/ESRGAN
  %cd {root}/stable-diffusion-webui/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
  print('Upscalers in Google Drive: %s'%upscalers_in_google_drive)
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # use lora model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/stable-diffusion-webui/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # use hypernetwork folder in google drive
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/stable-diffusion-webui/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # use VAE folder in google rive
  !mkdir -p {output_path}/VAE
  %cd {root}/stable-diffusion-webui/models
  !rm -rf VAE
  !ln -s {output_path}/VAE

In [ ]:
def applyGitFilemode():
  # git default file mode prevents checkout and fail
  print('Apply git filemode false')
  !cd {root}/stable-diffusion-webui/repositories/k-diffusion;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/taming-transformers;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/CodeFormer;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/BLIP;git config core.filemode false  
  
def cloneRepositories():
  !git clone https://github.com/crowsonkb/k-diffusion.git {root}/stable-diffusion-webui/repositories/k-diffusion
  !git clone https://github.com/Stability-AI/stablediffusion.git {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai
  !git clone https://github.com/CompVis/taming-transformers.git {root}/stable-diffusion-webui/repositories/taming-transformers
  !git clone https://github.com/sczhou/CodeFormer.git {root}/stable-diffusion-webui/repositories/CodeFormer
  !git clone https://github.com/salesforce/BLIP.git {root}/stable-diffusion-webui/repositories/BLIP
  

def installExtensionsFromURL(urls):
  %cd {root}/stable-diffusion-webui/extensions
  for url in urls.split(','):
    !git clone {url}

def lowRamPatch():
  !sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {root}/stable-diffusion-webui/launch.py

def deleteRepos():
  # delete repository directories in webui
  !rm -rf {root}/stable-diffusion-webui/repositories

In [ ]:
updatePython()

!mkdir -p {root}
os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

Already python 3.10. Skip install.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 122545 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.15.0-1ubuntu0.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.15.0-1ubuntu0.2) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.35.0-1build1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.35.0-1build1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.35.0-1build1_amd64.deb ...
Unpacking aria2 (1.35.0-1build1) ...
Setting up libc-ares2:amd64 (1.15.0-1ubuntu0.2) ...
Setting up libaria2-0:amd64 (1.35.0-1build1) ...
Setting up aria2 (1.35.0-1build1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for libc-bin (2.31-0ubuntu9.9) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 30.8 M

In [ ]:
if gMode == Everything:
  # delete existing repositories and reclone so the file mode fix can be applied
  # otherwise some will only be cloned in the final launch, causing some to fail to checkout.
  deleteRepos()
  cloneRepositories()
  applyGitFilemode()

# fix torch, torchvision version mismatch error
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

# A1111 first launch
%cd {root}/stable-diffusion-webui
!git checkout -f a9fed7c364061ae6efb37f797b6b522cb3cf7aa2
#!git checkout -f 22bcc7be428c94e9408f589966c2040187245d81
!COMMANDLINE_ARGS="--exit"  python launch.py


if gMode == Models_only:
  initSaveGoogleDriveModelOnly()

download_models()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 853.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 116.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu118
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling to

In [ ]:
installExtensionsFromURL(Extensions_from_URL)

# clear output
if Clear_Log:
  clear()




if  gMode != Everything:
  lowRamPatch()

%cd {root}/stable-diffusion-webui  
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --xformers'

if NGROK:
  args += f' --ngrok {NGROK} '
else:
  args += ' --share '
if username and password:
  args += f' --gradio-auth {username}:{password} '
print(f'WEBUI ARGUMENTS: {args}')
!python {root}/stable-diffusion-webui/launch.py {args}


/content/stable-diffusion-webui
WEBUI ARGUMENTS: --gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --xformers --share  --gradio-auth a:a 
Python 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
Commit hash: a9fed7c364061ae6efb37f797b6b522cb3cf7aa2
Installing xformers
Installing requirements for Web UI
Launching Web UI with arguments: --gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --xformers --share --gradio-auth a:a
2023-05-26 08:14:28.549934: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 08:14:30.839767: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Checkpoint 512-depth-ema.ckpt [ece79d931a] not found; loading 